DataScience - Modulo 4 - BIG Data - Tarea1 - Paulino Jose Moreno Solano

Comenzamos con la importación de librerías y demás parámetros necesarios

In [1]:
import IPython
IPython.auto_scroll_threshold = 9999

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

Nombre de la estructura o base de datos dentro del appName

In [3]:
import findspark
findspark.init('C:\Spark\spark-2.4.4-bin-hadoop2.7')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType
from pyspark.sql import SQLContext

Cargamos el archivo CSV con el set de datos

In [4]:
# File location and type
file_location = "nyc-jobs.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

DataFrame[Job ID: int, Agency: string, Posting Type: string, # Of Positions: int, Business Title: string, Civil Service Title: string, Title Code No: string, Level: string, Job Category: string, Full-Time/Part-Time indicator: string, Salary Range From: double, Salary Range To: double, Salary Frequency: string, Work Location: string, Division/Work Unit: string, Job Description: string, Minimum Qual Requirements: string, Preferred Skills: string, Additional Information: string, To Apply: string, Hours/Shift: string, Work Location 1: string, Recruitment Contact: string, Residency Requirement: string, Posting Date: string, Post Until: string, Posting Updated: string, Process Date: string]

In [5]:
df.count()

3326

In [6]:
len(df.columns)

28

Es necesario tener un vista de la estructura, para lo cual se requiere las columnas y del tipo de dato que guardan, para ver de una vez si es posible hacer operaciones matematicas o es texto. Este resumen se obtiene:

In [7]:
df.printSchema()

root
 |-- Job ID: integer (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Posting Type: string (nullable = true)
 |-- # Of Positions: integer (nullable = true)
 |-- Business Title: string (nullable = true)
 |-- Civil Service Title: string (nullable = true)
 |-- Title Code No: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Job Category: string (nullable = true)
 |-- Full-Time/Part-Time indicator: string (nullable = true)
 |-- Salary Range From: double (nullable = true)
 |-- Salary Range To: double (nullable = true)
 |-- Salary Frequency: string (nullable = true)
 |-- Work Location: string (nullable = true)
 |-- Division/Work Unit: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- Minimum Qual Requirements: string (nullable = true)
 |-- Preferred Skills: string (nullable = true)
 |-- Additional Information: string (nullable = true)
 |-- To Apply: string (nullable = true)
 |-- Hours/Shift: string (nullable = true)
 |-- Work Locat

Para tener un mejor conocimento de los datos, se requiere ver una parte del conjunto de datos, para lo cual con los primeros 20 registros de las columnas es suficiente, para lo cual usaremos la siguiente instruccion:

In [8]:
df.toPandas()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,None,None,...,â€¢\tExcellent interpersonal and organizationa...,"Salary range for this position is: $42,405 - $...",None,None,None,None,New York City residency is generally required ...,2011-06-24T00:00:00.000,None,2011-06-24T00:00:00.000
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,None,F,...,None,"In addition to applying through this website, ...",None,None,None,New York City residency is generally required ...,2012-01-26T00:00:00.000,None,2012-01-26T00:00:00.000,2019-09-03T00:00:00.000
2,132292,NYC HOUSING AUTHORITY,Internal,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,or construction trades in a trade school or v...,may be substituted for six months of the expe...,all candidates must have a minimum of two yea...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirements.
3,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,or construction trades in a trade school or v...,may be substituted for six months of the expe...,all candidates must have a minimum of two yea...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirements.
4,133921,NYC HOUSING AUTHORITY,Internal,50,Temporary Painter,PAINTER,91830,0,Maintenance & Operations,F,...,SPECIAL NOTE: 1. This is a temporary assig...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirement.,2014-01-09T00:00:00.000,None,2014-01-08T00:00:00.000,2019-09-03T00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,409915,HOUSING PRESERVATION & DVLPMNT,Internal,1,"Project Manager, Multifamily Preservation Prog...",HOUSING DEVELOPMENT SPECIALIST,22507,1,"Engineering, Architecture, & Planning",F,...,None,Apply Online.,None,100 Gold Street,None,New York City residency is generally required ...,2019-08-30T00:00:00.000,2019-09-29T00:00:00.000,2019-08-30T00:00:00.000,2019-09-03T00:00:00.000
3322,409970,ADMIN FOR CHILDREN'S SVCS,Internal,1,Operations Manager,ADMINISTRATIVE DIRECTOR OF RES,1001B,0,Social Services,F,...,The preferred candidate should possess the fol...,Section 424-A of the New York Social Services ...,"""Click on the """"Apply Now"""" button.""",None,None,None,New York City residency is generally required ...,2019-08-30T00:00:00.000,2019-09-13T00:00:00.000,2019-08-30T00:00:00.000
3323,410117,HRA/DEPT OF SOCIAL SERVICES,Internal,1,LEGAL SERVICES HOUSING COURT LIAISON,PRINCIPAL ADMINISTRATIVE ASSOC,10124,2,Social Services,F,...,"or """"3"""" above. However",all candidates must possess the one year of a...,None,**LOAN FORGIVENESS The federal government pro...,"""APPLICANTS MUST BE PERMANENT IN THE PRINCIPAL...",9:00 AM to 5:00 PM,None,None,New York City residency is generally required ...,2019-08-30T00:00:00.000
3324,410127,OFFICE OF THE COMPTROLLER,External,3,Contract Analyst - Contract and Procurement Re...,PROCUREMENT ANALYST,12158,2,"Finance, Accounting, & Procurement",F,...,after meeting the minimum requirements,either one year served at Assignment Level II...,at least one year of which must have been sup...,or spent performing professional procurement ...,â€¢ Experience reviewing complex contracts

Luego de ver los datos es conveniente un resumen pero de los datos, para lo cual usaremos una funcion que nos de ese resumen con un analisis de las columnas.

In [9]:
df.summary().toPandas()

,summary,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,count,3326,3326,3326,3326,3326,3326,3326,3326,3324,...,2691,3127,2070,2015,1295,2539,2779,1750,2763,2886
1,mean,375806.9239326518,None,None,2.4690318701142515,None,None,35215.61260396673,1.0190441969098096,None,...,None,None,None,None,None,None,None,None,None,None
2,stddev,46518.01232571711,None,None,7.832308454350913,None,None,27677.86960089443,1.1058290522164882,None,...,None,None,None,None,None,None,None,None,None,None
3,min,87990,ADMIN FOR CHILDREN'S SVCS,External,1,.NET Developer,ACCOUNTANT,0527A,0,Administration & Human Resources,...,all candidates must have a four-year high s...,"""""2""""","""""2""""","""""2""""",Environmental Health,executive or supervisory capacity. The super...,"""""2"""" or """"3"""" above. However","""""2""""","""""2""""",architecture
4,25%,374967,None,None,1,None,None,12627.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
5,50%,394896,None,None,1,None,None,22427.0,1.0,None,...,None,None,None,None,None,None,None,None,None,None
6,75%,402418,None,None,1,None,None,56057.0,2.0,None,...,None,None,None,None,None,None,None,None,None,None
7,max,410127,TEACHERS RETIREMENT SYSTEM,Internal,150,iOS Mobile Application Developer,YOUTH DEVELOPMENT SPECIALIST,95725,M8,"Technology, Data & Innovation Social Services",...,â€“Masterâ€™s degree from an accredited colleg...,â€¢Extensive experience with CommVault Simpana...,"â€¢Litigation experience, including in New Yor...",â€¢5 years Microsoft SQL Server administration...,â€¢ Familiarity with the Cityâ€™s procurement ...,â€œNOTE: This position is open to qualified pe...,â€¢Strong project and program management and l...,â€œNOTE: This position is open to qualified pe...,â€¢ A valid or ability to obtain a New York St...,â€¢ Driverâ€™s license valid in the State of N...


En algunos set de datos los nombres de las columnas vienen con algunos caracteres que nos pueden dificultar el incorporar ese nombre de columna con el codigo, o bien es necesario ajustar el nombre para tener una mejor referencia de los datos o bien de la columna, para este efecto usaremos esta logica para ajustar el nombre de todas las columas.

In [10]:
newcolnames = ['Job_ID','Agency','Posting_Type','No_Of_Positions','Business_Title','Civil_Service_Title','Title_Code_No','Level','Job_Category','Full_Time_by_Part_Time_indicator','Salary_Range_From','Salary_Range_To','Salary_Frequency','Work_Location','Division_by_Work_Unit','Job_Description','Minimum_Qual_Requirements','Preferred_Skills','Additional_Information','To_Apply','Hours_by_Shift','Work_Location_1','Recruitment_Contact','Residency_Requirement','Posting_Date','Post_Until','Posting_Updated','Process_Date']

#Replace All columns
for c,n in zip(df.columns,newcolnames):
    df=df.withColumnRenamed(c,n)

df.toPandas()

,Job_ID,Agency,Posting_Type,No_Of_Positions,Business_Title,Civil_Service_Title,Title_Code_No,Level,Job_Category,Full_Time_by_Part_Time_indicator,...,Additional_Information,To_Apply,Hours_by_Shift,Work_Location_1,Recruitment_Contact,Residency_Requirement,Posting_Date,Post_Until,Posting_Updated,Process_Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,None,None,...,â€¢\tExcellent interpersonal and organizationa...,"Salary range for this position is: $42,405 - $...",None,None,None,None,New York City residency is generally required ...,2011-06-24T00:00:00.000,None,2011-06-24T00:00:00.000
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,None,F,...,None,"In addition to applying through this website, ...",None,None,None,New York City residency is generally required ...,2012-01-26T00:00:00.000,None,2012-01-26T00:00:00.000,2019-09-03T00:00:00.000
2,132292,NYC HOUSING AUTHORITY,Internal,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,or construction trades in a trade school or v...,may be substituted for six months of the expe...,all candidates must have a minimum of two yea...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirements.
3,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,...,or construction trades in a trade school or v...,may be substituted for six months of the expe...,all candidates must have a minimum of two yea...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirements.
4,133921,NYC HOUSING AUTHORITY,Internal,50,Temporary Painter,PAINTER,91830,0,Maintenance & Operations,F,...,SPECIAL NOTE: 1. This is a temporary assig...,"""Click the """"Apply Now"""" button.""",None,None,None,NYCHA has no residency requirement.,2014-01-09T00:00:00.000,None,2014-01-08T00:00:00.000,2019-09-03T00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,409915,HOUSING PRESERVATION & DVLPMNT,Internal,1,"Project Manager, Multifamily Preservation Prog...",HOUSING DEVELOPMENT SPECIALIST,22507,1,"Engineering, Architecture, & Planning",F,...,None,Apply Online.,None,100 Gold Street,None,New York City residency is generally required ...,2019-08-30T00:00:00.000,2019-09-29T00:00:00.000,2019-08-30T00:00:00.000,2019-09-03T00:00:00.000
3322,409970,ADMIN FOR CHILDREN'S SVCS,Internal,1,Operations Manager,ADMINISTRATIVE DIRECTOR OF RES,1001B,0,Social Services,F,...,The preferred candidate should possess the fol...,Section 424-A of the New York Social Services ...,"""Click on the """"Apply Now"""" button.""",None,None,None,New York City residency is generally required ...,2019-08-30T00:00:00.000,2019-09-13T00:00:00.000,2019-08-30T00:00:00.000
3323,410117,HRA/DEPT OF SOCIAL SERVICES,Internal,1,LEGAL SERVICES HOUSING COURT LIAISON,PRINCIPAL ADMINISTRATIVE ASSOC,10124,2,Social Services,F,...,"or """"3"""" above. However",all candidates must possess the one year of a...,None,**LOAN FORGIVENESS The federal government pro...,"""APPLICANTS MUST BE PERMANENT IN THE PRINCIPAL...",9:00 AM to 5:00 PM,None,None,New York City residency is generally required ...,2019-08-30T00:00:00.000
3324,410127,OFFICE OF THE COMPTROLLER,External,3,Contract Analyst - Contract and Procurement Re...,PROCUREMENT ANALYST,12158,2,"Finance, Accounting, & Procurement",F,...,after meeting the minimum requirements,either one year served at Assignment Level II...,at least one year of which must have been sup...,or spent performing professional procurement ...,â€¢ Experience reviewing complex co

En un analisis preliminar pordemos scar un pareto o 80/20, para identificar en el caso de las Agencias, las que nos den el mayor volumen de datos, que en estas agencias que conforman ese 80% es donde estan comprendidos la mayoria de los trabajos, este analises se puede obtener usando estos pasos:

In [11]:
#Create a View
df.createOrReplaceTempView("AgrupacionSalario")

#SQL Paretto Analisys
sqlDF = spark.sql("""select *, (JobsAgency /total)*100 as Avg, (subTotal /total)*100 as AvgTotal from (
                        select *, sum(JobsAgency) over (order by JobsAgency desc, Agency) as subTotal, sum(JobsAgency) over () as total
                        from (SELECT Agency,  Count(0) as JobsAgency FROM AgrupacionSalario group by Agency) as datos
                        ) as agrupacion
                """)
sqlDF.toPandas()

,Agency,JobsAgency,subTotal,total,Avg,AvgTotal
0,DEPT OF ENVIRONMENT PROTECTION,724,724,3326,21.767889,21.767889
1,DEPT OF HEALTH/MENTAL HYGIENE,275,999,3326,8.268190,30.036079
2,NYC HOUSING AUTHORITY,216,1215,3326,6.494287,36.530367
3,DEPT OF DESIGN & CONSTRUCTION,205,1420,3326,6.163560,42.693927
4,DEPARTMENT OF TRANSPORTATION,199,1619,3326,5.983163,48.677090
5,DEPT OF INFO TECH & TELECOMM,153,1772,3326,4.600120,53.277210
6,HOUSING PRESERVATION & DVLPMNT,122,1894,3326,3.668070,56.945280
7,ADMIN FOR CHILDREN'S SVCS,102,1996,3326,3.066747,60.012026
8,TAXI & LIMOUSINE COMMISSION,93,2089,3326,2.796152,62.808178
9,DEPT OF PARKS & RECREATION,91,2180,3326,2.736019,65.544197


En algunos casos es necesario ingresar algunos datos que nos ayuden a "resumir" la data o agruparla de alguna manera, para lo cual es conveniente ingresar una nueva columna con un rango de datos (estos rangos salen de otro analisis, luego de ver la distribucion de los salarios, se opta por seleccionar una escalera de 25k$ entre cada item), esto puede ser la base para un analisis como el pareto que vimos anteriormente. Para agregar una nueva columna con un rango podemos usar estos pasos.

In [12]:
#Create a View
df.createOrReplaceTempView("AgrupacionSalario")

#SQL Add new Range Column
sqlDF = spark.sql("""select Nueva, Count(0) as JobsCount
                        from (SELECT
                                    case 
                                    when Salary_Range_From < 25000 then '0-25000'
                                    when Salary_Range_From < 50000 then '25000-50000'
                                    when Salary_Range_From < 75000 then '50000-75000'
                                    when Salary_Range_From < 100000 then '75000-100000'
                                    when Salary_Range_From < 125000 then '100000-125000'
                                    when Salary_Range_From < 150000 then '125000-150000'
                                    when Salary_Range_From < 175000 then '150000-175000'
                                    when Salary_Range_From < 200000 then '175000-200000'
                                    when Salary_Range_From < 225000 then '200000-225000'
                                    else '225000+'
                                    end as Nueva
                                , *
                            FROM AgrupacionSalario) as datos
                        group by Nueva
                        order by Nueva
                """)
sqlDF.toPandas()

,Nueva,JobsCount
0,0-25000,281
1,100000-125000,84
2,125000-150000,21
3,150000-175000,6
4,175000-200000,8
5,200000-225000,6
6,25000-50000,645
7,50000-75000,1723
8,75000-100000,552
